In [1]:
seed = 42  # for reproducibility

# Imports
import os
import sys
import enum
import copy
import random
import tempfile
import warnings
import multiprocessing
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display

# import visdom
from pathlib import Path
from tqdm import tqdm
from scipy import stats

import torch
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image
torch.manual_seed(seed)
import torchio as tio
sys.path.append(str(Path().absolute().parent))

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



In [2]:
DDSM_mass_path = Path('/home/linardos/Datasets/DDSM/Images/CBIS-DDSM/Mass')
train_path = DDSM_mass_path.joinpath('Train')

In [3]:
csv_file = pd.read_csv(Path('/home/linardos/Datasets/DDSM/mass_case_description_test_set.csv'))
new_csv_file = pd.read_csv(Path('/home/linardos/Datasets/DDSM/mass_case_description_test_set.csv'))

In [4]:
csv_file

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00016_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Mass-Test_P_00016_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00016_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Mass-Test_P_00017_LEFT_CC_1/1.3.6.1.4.1.9590.1...
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,Mass-Test_P_00017_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_00017_LEFT_MLO_1/1.3.6.1.4.1.9590....
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,Mass-Test_P_00032_RIGHT_CC/1.3.6.1.4.1.9590.10...,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_00032_RIGHT_CC_1/1.3.6.1.4.1.9590....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,P_01825,2,RIGHT,MLO,1,mass,LOBULATED,MICROLOBULATED,3,BENIGN_WITHOUT_CALLBACK,3,Mass-Test_P_01825_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Mass-Test_P_01825_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Mass-Test_P_01825_RIGHT_MLO_1/1.3.6.1.4.1.9590...
374,P_01833,2,RIGHT,MLO,1,mass,IRREGULAR,ILL_DEFINED,5,MALIGNANT,5,Mass-Test_P_01833_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Mass-Test_P_01833_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Mass-Test_P_01833_RIGHT_MLO_1/1.3.6.1.4.1.9590...
375,P_01865,2,LEFT,MLO,1,mass,IRREGULAR,ILL_DEFINED,4,MALIGNANT,2,Mass-Test_P_01865_LEFT_MLO/1.3.6.1.4.1.9590.10...,Mass-Test_P_01865_LEFT_MLO_1/1.3.6.1.4.1.9590....,Mass-Test_P_01865_LEFT_MLO_1/1.3.6.1.4.1.9590....
376,P_01912,3,RIGHT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,4,Mass-Test_P_01912_RIGHT_CC/1.3.6.1.4.1.9590.10...,Mass-Test_P_01912_RIGHT_CC_1/1.3.6.1.4.1.9590....,Mass-Test_P_01912_RIGHT_CC_1/1.3.6.1.4.1.9590....


In [5]:
csv_file['image file path'][0]

'Mass-Test_P_00016_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.416403281812750683720028031170500130104/1.3.6.1.4.1.9590.100.1.2.245063149211255120613007755642780114172/000000.dcm'

In [6]:
csv_file['image file path'][5]

'Mass-Test_P_00032_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.383460844111861145315621774372145221963/1.3.6.1.4.1.9590.100.1.2.318799084911119262430780458250312419361/000000.dcm'

In [7]:
new_csv_file['image file path'] = csv_file['image file path'].str.replace('Mass-Test_','')
new_csv_file['image file path'] = new_csv_file['image file path'].str.split('/', expand=True)[0]
new_csv_file['image file path'] = 'Test/' + new_csv_file['image file path'].astype(str) + '/scan.nii.gz'
new_csv_file['cropped image file path'] = csv_file['cropped image file path'].str.replace('Mass-Test_','')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.split('/', expand=True)[0]
new_csv_file['cropped image file path'] = 'Test/' + new_csv_file['cropped image file path'].astype(str) + '/roi_mass.nii.gz'
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_1/r','/r')
new_csv_file['ROI mask file path'] = csv_file['ROI mask file path'].str.replace('Mass-Test_','')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.split('/', expand=True)[0]
new_csv_file['ROI mask file path'] = 'Test/' + new_csv_file['ROI mask file path'].astype(str) + '/mask_mass.nii.gz'
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_1/m','/m')

new_csv_file['image file path'] = new_csv_file['image file path'].str.replace('_LEFT','/LEFT')
new_csv_file['image file path'] = new_csv_file['image file path'].str.replace('_RIGHT','/RIGHT')

new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_LEFT','/LEFT')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_RIGHT','/RIGHT')

new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_LEFT','/LEFT')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_RIGHT','/RIGHT')

In [8]:
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'CC/roi_mass.nii',f'CC/roi_mass_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'CC/mask_mass.nii',f'CC/mask_mass_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'MLO/roi_mass.nii',f'MLO/roi_mass_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'MLO/mask_mass.nii',f'MLO/mask_mass_1.nii')

new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'CC/roi_mass.nii',f'CC/roi_mass_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'CC/mask_mass.nii',f'CC/mask_mass_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'MLO/roi_mass.nii',f'MLO/roi_mass_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'MLO/mask_mass.nii',f'MLO/mask_mass_1.nii')

for i in range(20):
    new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'_{i}/roi_mass.nii',f'/roi_mass_{i}.nii')
    new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'_{i}/mask_mass.nii',f'/mask_mass_{i}.nii')

new_csv_file.columns = new_csv_file.columns.str.replace(' ','_')

In [9]:
new_csv_file

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,mass_shape,mass_margins,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Test/P_00016/LEFT_CC/scan.nii.gz,Test/P_00016/LEFT_CC/roi_mass_1.nii.gz,Test/P_00016/LEFT_CC/mask_mass_1.nii.gz
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,Test/P_00016/LEFT_MLO/scan.nii.gz,Test/P_00016/LEFT_MLO/roi_mass_1.nii.gz,Test/P_00016/LEFT_MLO/mask_mass_1.nii.gz
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,Test/P_00017/LEFT_CC/scan.nii.gz,Test/P_00017/LEFT_CC/roi_mass_1.nii.gz,Test/P_00017/LEFT_CC/mask_mass_1.nii.gz
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,Test/P_00017/LEFT_MLO/scan.nii.gz,Test/P_00017/LEFT_MLO/roi_mass_1.nii.gz,Test/P_00017/LEFT_MLO/mask_mass_1.nii.gz
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,Test/P_00032/RIGHT_CC/scan.nii.gz,Test/P_00032/RIGHT_CC/roi_mass_1.nii.gz,Test/P_00032/RIGHT_CC/mask_mass_1.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,P_01825,2,RIGHT,MLO,1,mass,LOBULATED,MICROLOBULATED,3,BENIGN_WITHOUT_CALLBACK,3,Test/P_01825/RIGHT_MLO/scan.nii.gz,Test/P_01825/RIGHT_MLO/roi_mass_1.nii.gz,Test/P_01825/RIGHT_MLO/mask_mass_1.nii.gz
374,P_01833,2,RIGHT,MLO,1,mass,IRREGULAR,ILL_DEFINED,5,MALIGNANT,5,Test/P_01833/RIGHT_MLO/scan.nii.gz,Test/P_01833/RIGHT_MLO/roi_mass_1.nii.gz,Test/P_01833/RIGHT_MLO/mask_mass_1.nii.gz
375,P_01865,2,LEFT,MLO,1,mass,IRREGULAR,ILL_DEFINED,4,MALIGNANT,2,Test/P_01865/LEFT_MLO/scan.nii.gz,Test/P_01865/LEFT_MLO/roi_mass_1.nii.gz,Test/P_01865/LEFT_MLO/mask_mass_1.nii.gz
376,P_01912,3,RIGHT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,4,Test/P_01912/RIGHT_CC/scan.nii.gz,Test/P_01912/RIGHT_CC/roi_mass_1.nii.gz,Test/P_01912/RIGHT_CC/mask_mass_1.nii.gz


In [10]:
new_csv_file.to_csv('/home/linardos/Datasets/DDSM/processed_mass_case_description_test_set.csv')